In [1]:
import os
import subprocess
from joblib import Parallel, delayed

from tqdm import tqdm


In [2]:
def create_pdf_from_latex(latex_content: str, output_filename:str, counter: int) -> bool:
    # Check if the content is already a math environment
    if latex_content.strip().startswith("\\begin"):
        math_content = latex_content
    else:
        math_content = f"\\({latex_content}\\)".replace("\\ \\", "\\\\")  # Inline math

    document_template = f"""
    \\documentclass[12pt]{{article}}
    \\usepackage{{amsmath}}
    \\usepackage{{amsfonts}}
    \\usepackage{{amssymb}}
    \\usepackage{{amsthm}}
    \\usepackage{{mathtools}}
    \\usepackage{{physics}}
    \\usepackage{{siunitx}}
    \\usepackage{{mhchem}}
    \\pagestyle{{empty}}
    \\begin{{document}}
    \\begin{{center}}
    {math_content}
    \\end{{center}}
    \\end{{document}}
    """
    
    try: 
        with open(f"temp_{counter}.tex", "w") as file:
            file.write(document_template)
    
        subprocess.run(["pdflatex", f"temp_{counter}.tex"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        os.rename(f"temp_{counter}.pdf", f"{output_filename}")
        os.remove(f"temp_{counter}.tex")
        os.remove(f"temp_{counter}.aux")
        os.remove(f"temp_{counter}.log")
        
        result = subprocess.run(["pdfcrop", 
                                 output_filename, 
                                 output_filename], capture_output=True, text=True)
        return True
    except Exception as e:
        return False

# Example usage
latex_math = open('equations/equations.txt','r').readlines()[18]
output_file = "output_0.pdf"

create_pdf_from_latex(latex_math, output_file, 0)


True

In [7]:
equations = open('equations/unique_equations.txt','r').readlines()
rendered_equations = []

In [ ]:
# Render equations into .pdf 's 
counter = 0
with open('outputs/rendered_equations.txt','w') as file:
    for equation in tqdm(equations):
        if create_pdf_from_latex(equation, 'outputs/output_' + str(counter) + '.pdf', counter):
            rendered_equations.append(equation)
            file.write(equation)
        counter += 1


In [3]:

# Clean-up
for filename in os.listdir('./'):
    if filename.endswith('.aux') or filename.endswith('.tex') or filename.endswith('log'):
        os.remove(filename)
    

In [37]:
# Change .pdf renders to .png renders
for file in tqdm(os.listdir('outputs')):
    if file.endswith('.pdf'):
        result = subprocess.run(["convert",
                                 "-density",
                                 "500",
                                 'outputs/'+file,
                                 'outputs/'+file[:-4]+".png"])
        os.remove('outputs/'+file)